# Acquire

**Goal: Your goal is to predict the values of single unit properties using the obervations from 2017.**

**import**

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys
from sklearn.model_selection import train_test_split

sys.path.append("./util_")
import acquire_
import prepare_


**get data**

In [2]:
# sql query
query = """
SELECT bedroomcnt, 
		bathroomcnt,
        calculatedfinishedsquarefeet,
        taxvaluedollarcnt,
        yearbuilt,
        taxamount,
        fips
FROM properties_2017
WHERE propertylandusetypeid = 261 -- Single family home
"""

# UNION

# SELECT bedroomcnt, 
# 		bathroomcnt,
#         calculatedfinishedsquarefeet,
#         taxvaluedollarcnt,
#         yearbuilt,
#         taxamount,
#         fips
# FROM properties_2017
# WHERE propertylandusetypeid = 261 -- Single family home;
# """

In [3]:
# get data from codeup database
zillow, q = acquire_.get_codeup_sql_data_(db_name="zillow", query=query,fileName="zillow_single_family")

## Understand data

In [4]:
zillow.shape

(2152863, 7)

In [5]:
zillow.columns

Index(['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet',
       'taxvaluedollarcnt', 'yearbuilt', 'taxamount', 'fips'],
      dtype='object')

In [6]:
zillow.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152852 non-null  float64
 1   bathroomcnt                   2152852 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152370 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 115.0 MB


In [7]:
zillow.describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
count,2.152852e+06,2.152852e+06,2.144379e+06,2.152370e+06,2.143526e+06,2.148421e+06,2.152863e+06
mean,3.287196e+00,2.230688e+00,1.862855e+03,4.618962e+05,1.960950e+03,5.634866e+03,6.048377e+03
std,9.547544e-01,9.992796e-01,1.222125e+03,6.996760e+05,2.216220e+01,8.178910e+03,2.043329e+01
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.801000e+03,1.850000e+00,6.037000e+03
25%,3.000000e+00,2.000000e+00,1.257000e+03,1.881702e+05,1.949000e+03,2.534980e+03,6.037000e+03
50%,3.000000e+00,2.000000e+00,1.623000e+03,3.276710e+05,1.958000e+03,4.108950e+03,6.037000e+03
75%,4.000000e+00,3.000000e+00,2.208000e+03,5.345270e+05,1.976000e+03,6.414320e+03,6.059000e+03
max,2.500000e+01,3.200000e+01,9.525760e+05,9.842891e+07,2.016000e+03,1.337756e+06,6.111000e+03


In [8]:
# count of numeric columns and object columns
numerics = len(zillow.select_dtypes("number").columns)
objects = len(zillow.select_dtypes("object").columns)

print("Numeric col count:", numerics)
print("object col count:", objects)

Numeric col count: 7
object col count: 0


**What I see:**

- I have 2152863 rows and 7 columns
- 7 of the 29 columns are numric while 0 of them are string object colums
- I have unsure null values in my data
- I also see the descriptive statistics of my data

# Prepare

In [9]:
zillow.T

,0,1,2,3,4,5,6,7,8,9,...,2152853,2152854,2152855,2152856,2152857,2152858,2152859,2152860,2152861,2152862
bedroomcnt,0.0,0.0,0.0,0.00,4.00,0.0,3.00,3.00,0.00,0.00,...,4.00,0.0,3.0,4.00,0.00,4.00,4.00,0.00,3.0,4.0
bathroomcnt,0.0,0.0,0.0,0.00,2.00,0.0,4.00,2.00,0.00,0.00,...,2.00,0.0,2.5,4.00,0.00,3.00,4.50,0.00,2.5,4.0
calculatedfinishedsquarefeet,NaN,NaN,NaN,NaN,3633.00,NaN,1620.00,2077.00,NaN,NaN,...,1987.00,NaN,1809.0,4375.00,NaN,2262.00,3127.00,NaN,1974.0,2110.0
taxvaluedollarcnt,27516.0,10.0,10.0,2108.00,296425.00,124.0,847770.00,646760.00,6730242.00,15532.00,...,259913.00,1198476.0,405547.0,422400.00,1087111.00,960756.00,536061.00,208057.00,424353.0,554009.0
yearbuilt,NaN,NaN,NaN,NaN,2005.00,NaN,2011.00,1926.00,NaN,NaN,...,1955.00,NaN,2012.0,2015.00,NaN,2015.00,2014.00,NaN,2015.0,2014.0
taxamount,NaN,NaN,NaN,174.21,6941.39,NaN,10244.94,7924.68,80348.13,248.89,...,3175.66,NaN,4181.1,13877.56,19313.08,13494.52,6244.16,5783.88,5302.7,6761.2
fips,6037.0,6037.0,6037.0,6037.00,6037.00,6037.0,6037.00,6037.00,6037.00,6037.00,...,6059.00,6037.0,6059.0,6037.00,6059.00,6059.00,6059.00,6059.00,6059.0,6037.0


**Rename columns**

In [10]:
# rename dataframe columns
zillow = zillow.rename(columns={"bedroomcnt":"bedrooms",
                       "bathroomcnt":"bathrooms",
                       "calculatedfinishedsquarefeet":"sqr_feet",
                      "taxvaluedollarcnt":"tax_value",
                      "yearbuilt":"year_built",
                      "taxamount":"tax_amount",
                      "fips":"county"})

**drop nulls**

In [11]:
# drop all nulls in the dataframe
zillow = zillow.dropna()

**convert data type**

In [12]:
# convert data type from float to int
zillow.bedrooms = zillow.bedrooms.astype(int)
zillow.year_built = zillow.year_built.astype(int)

**Remove duplicated rows**

In [13]:
# remove the duplocated rows
zillow = zillow.drop_duplicates(keep="first")

**Remove outliers**

This is done using box plot in the explore phase. only looking at the training data.

In [14]:
# remove outliers
zillow = zillow[zillow.bedrooms <= 7]
zillow = zillow[zillow.bathrooms <= 7]
zillow = zillow[zillow.year_built >= 1900]
zillow = zillow[zillow.sqr_feet <= 5000]
zillow = zillow[zillow.tax_amount <= 20000]

**feature engineeing**

In [15]:
# Rename the unique values in fips to county names
zillow.county = zillow.county.astype(str).str.replace("6037.0","Los Angeles").str.replace("6059.0","Orange").str.replace("6111.0","Sam Juan")


## Split

In [16]:
# split the data into training, validation and testing sets
train, validate, test = prepare_.split_data_(df=zillow,
                    test_size=0.2, 
                     validate_size=0.2, 
                     random_state=95)
(train.shape, validate.shape, test.shape)

((1237902, 7), (412634, 7), (412635, 7))

**Save split**

In [17]:
prepare_.save_split_data(encoded_df=zillow, train=train, validate=validate, test=test)

'Four data sets saved as .csv'